In [30]:
# general 
import pandas as pd 
import numpy as np 
from datetime import datetime

# stats
from scipy import stats

# plotting
import seaborn as sns 
import plotly.express as px
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# settings
plt.style.use('seaborn')
plt.rcParams["figure.figsize"] = (16, 8)
pd.options.plotting.backend = "plotly"

# ML
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [31]:
hrv_file = "04-12-21-HRV-Data-Postprocess-Kubios.csv"
cgm_file = "04-12-21-CGM-Data.csv"

## HRV Analysis
The daily HRV file from 04/12/21 is provided here and prepared for analysis. Note that unlike the CGM values which are of 24 hour, HRV is only collected as per:

- 5 minutes before breakfast
- 2 hours post dinner

In most cases, the timings for collection will vary. In this case, collected was started at 0731 hours and concluded at 2209 hours.

In [32]:
# parse HRV file  
df = pd.read_csv(hrv_file,
                     parse_dates={'date-time' : ['time', "date"]},
                     index_col='date-time'
                     )

df.drop(['seconds'], axis=1, inplace=True)
print(df)

                       rr
date-time                
2012-12-04 07:31:37   855
2012-12-04 07:31:38   855
2012-12-04 07:31:39   862
2012-12-04 07:31:40   890
2012-12-04 07:31:41   950
...                   ...
2012-12-04 22:09:13  1185
2012-12-04 22:09:15  1239
2012-12-04 22:09:16  1275
2012-12-04 22:09:17  1225
2012-12-04 22:09:18  1291

[66996 rows x 1 columns]


##### Observation
Because HRV data is the beat to beat interval, some portions are >1 sec. This means that from 7.31.37am to 22.09.18pm, there are more than 52661 seconds (14 hours converted to seconds).

Here, 66996 rows are observed. We proceed to resample it to fit 52661 seconds.

In [33]:
# resample into 1 sec intervals, pad in between values
df = df.resample('1s').median()

# some NaN values are observed 
print(df)

                         rr
date-time                  
2012-12-04 07:31:37   855.0
2012-12-04 07:31:38   855.0
2012-12-04 07:31:39   862.0
2012-12-04 07:31:40   890.0
2012-12-04 07:31:41   919.0
...                     ...
2012-12-04 22:09:14     NaN
2012-12-04 22:09:15  1239.0
2012-12-04 22:09:16  1275.0
2012-12-04 22:09:17  1225.0
2012-12-04 22:09:18  1291.0

[52662 rows x 1 columns]


In [34]:
# check for and replace NaN values
df['rr'].isnull().sum()

2232

In [35]:
df.interpolate(method='linear', inplace=True)

In [36]:
fig = px.line(
    df,
    x=df.index,
    y="rr"
)
fig.show()

##### HRV Metrics (RMSSD, SDNN, etc)
Because readings are raw HRV values, a better approach would be to calculate RR intervals on a 2 minute epoch.

A python library: hrv-analysis (https://pypi.org/project/hrv-analysis/) is used.

In [37]:
from hrvanalysis import get_time_domain_features

In [38]:
# chunk the data into 2 minute segments (1 seconds)
def to_two_minute_chunks(values, segment_size=120):
    return [values[i:i+segment_size] for i in range(0,len(values),segment_size)]

chunked_rr = to_two_minute_chunks(df['rr'], segment_size=120)
chunked_rr[:1]

[date-time
 2012-12-04 07:31:37    855.0
 2012-12-04 07:31:38    855.0
 2012-12-04 07:31:39    862.0
 2012-12-04 07:31:40    890.0
 2012-12-04 07:31:41    919.0
                        ...  
 2012-12-04 07:33:32    722.0
 2012-12-04 07:33:33    720.0
 2012-12-04 07:33:34    705.0
 2012-12-04 07:33:35    702.0
 2012-12-04 07:33:36    673.0
 Freq: S, Name: rr, Length: 120, dtype: float64]

In [39]:
chunked_rr_metrics = list(map(get_time_domain_features, chunked_rr))
chunked_rr_metrics[:1]

[{'mean_nni': 833.1458333333334,
  'sdnn': 111.85797935560164,
  'sdsd': 45.55476407474942,
  'nni_50': 17,
  'pnni_50': 14.285714285714286,
  'nni_20': 50,
  'pnni_20': 42.016806722689076,
  'rmssd': 45.58043034298931,
  'median_nni': 799.0,
  'range_nni': 449.0,
  'cvsd': 0.05470882589741409,
  'cvnni': 0.13425978367804953,
  'mean_hr': 73.23584304383654,
  'max_hr': 89.95502248875562,
  'min_hr': 53.763440860215056,
  'std_hr': 9.199179611947093}]

In [40]:
dti = pd.date_range(start="2021-12-04 07:31:37",
                    end="2021-12-04 22:09:18",
                    freq="2min")
dti

DatetimeIndex(['2021-12-04 07:31:37', '2021-12-04 07:33:37',
               '2021-12-04 07:35:37', '2021-12-04 07:37:37',
               '2021-12-04 07:39:37', '2021-12-04 07:41:37',
               '2021-12-04 07:43:37', '2021-12-04 07:45:37',
               '2021-12-04 07:47:37', '2021-12-04 07:49:37',
               ...
               '2021-12-04 21:49:37', '2021-12-04 21:51:37',
               '2021-12-04 21:53:37', '2021-12-04 21:55:37',
               '2021-12-04 21:57:37', '2021-12-04 21:59:37',
               '2021-12-04 22:01:37', '2021-12-04 22:03:37',
               '2021-12-04 22:05:37', '2021-12-04 22:07:37'],
              dtype='datetime64[ns]', length=439, freq='2T')

In [41]:
all_hrv_metrics = pd.DataFrame([_ for _ in chunked_rr_metrics])
all_hrv_metrics['date-time'] = dti
all_hrv_metrics.set_index('date-time', inplace=True)

In [42]:
# visualize HRV metrics of interest
all_hrv_metrics.plot(
        all_hrv_metrics.index,
        ["sdnn", "rmssd", "pnni_50"],
    )

In [43]:
# upsample to 1 minute buckets to match CGM data
hrv_df = all_hrv_metrics.resample('1T').median().interpolate()
hrv_df

,mean_nni,sdnn,sdsd,nni_50,pnni_50,nni_20,pnni_20,rmssd,median_nni,range_nni,cvsd,cvnni,mean_hr,max_hr,min_hr,std_hr
date-time,,,,,,,,,,,,,,,,
2021-12-04 07:31:00,833.145833,111.857979,45.554764,17.0,14.285714,50.0,42.016807,45.580430,799.00,449.00,0.054709,0.134260,73.235843,89.955022,53.763441,9.199180
2021-12-04 07:32:00,829.362500,95.799753,35.139770,12.0,10.084034,47.0,39.495798,35.179309,820.50,399.50,0.042361,0.115424,73.301489,89.955022,56.380246,8.214715
2021-12-04 07:33:00,825.579167,79.741528,24.724775,7.0,5.882353,44.0,36.974790,24.778187,842.00,350.00,0.030013,0.096589,73.367134,89.955022,58.997050,7.230250
2021-12-04 07:34:00,677.605208,111.529095,26.672081,6.5,5.462185,28.5,23.949580,26.860841,657.00,472.75,0.042331,0.183593,96.228441,115.317488,58.852733,14.630706
2021-12-04 07:35:00,529.631250,143.316663,28.619386,6.0,5.042017,13.0,10.924370,28.943494,472.00,595.50,0.054648,0.270597,119.089747,140.679953,58.708415,22.031161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-04 22:03:00,1092.537500,194.812862,44.162433,28.0,23.529412,72.0,60.504202,44.381416,1177.50,607.00,0.040622,0.178312,57.151129,86.206897,46.047583,12.649581
2021-12-04 22:04:00,1161.854167,134.469976,41.109496,22.0,18.487395,62.0,52.100840,41.310218,1212.00,565.00,0.035841,0.119260,53.056762,81.174514,45.907086,8.254712
2021-12-04 22:05:00,1231.170833,74.127091,38.056558,16.0,13.445378,52.0,43.697479,38.239021,1246.50,523.00,0.031059,0.060209,48.962394,76.142132,45.766590,3.859843


## CGM Analysis
The daily CGM file from the 04/12/21 is provided here and prepared for analysis.

In [44]:
df = pd.read_csv(cgm_file,
                 parse_dates={'date-time': ['date','time']},
                 index_col ="date-time")

df.drop(['device', 'serial number'], axis=1, inplace=True)
df.head()

,glucose
date-time,
2021-12-04 00:00:00,5.4
2021-12-04 00:15:00,5.3
2021-12-04 00:30:00,5.1
2021-12-04 00:45:00,4.7
2021-12-04 01:00:00,4.3


#### Observations

HRV starts from 27097 seconds (7.31am). CGM data starts from 12am. Slicing of CGM time series is necessitated for easier comparison.

In [45]:
# resample into 1 min intervals, interpolate in-between values
df = df.resample('1T').interpolate()
cgm_df = df["2021-12-04 07:31:00":"2021-12-04 22:07:00"]
cgm_df

,glucose
date-time,
2021-12-04 07:31:00,5.733333
2021-12-04 07:32:00,5.800000
2021-12-04 07:33:00,5.793333
2021-12-04 07:34:00,5.786667
2021-12-04 07:35:00,5.780000
...,...
2021-12-04 22:03:00,6.260000
2021-12-04 22:04:00,6.273333
2021-12-04 22:05:00,6.286667


In [46]:
# no NaN values in CGM data
cgm_df['glucose'].isnull().sum()

0

In [47]:
fig = px.line(
    cgm_df,
    x=cgm_df.index,
    y='glucose'
)
fig.show()

At this point, both CGM and HRV data have the same timestamps (1 sec intervals) and can be merged

In [48]:
df = pd.concat([cgm_df.reset_index(), hrv_df.reset_index()], axis=1)
df = df.loc[:,~df.columns.duplicated()]
df.set_index('date-time', inplace=True)
df

,glucose,mean_nni,sdnn,sdsd,nni_50,pnni_50,nni_20,pnni_20,rmssd,median_nni,range_nni,cvsd,cvnni,mean_hr,max_hr,min_hr,std_hr
date-time,,,,,,,,,,,,,,,,,
2021-12-04 07:31:00,5.733333,833.145833,111.857979,45.554764,17.0,14.285714,50.0,42.016807,45.580430,799.00,449.00,0.054709,0.134260,73.235843,89.955022,53.763441,9.199180
2021-12-04 07:32:00,5.800000,829.362500,95.799753,35.139770,12.0,10.084034,47.0,39.495798,35.179309,820.50,399.50,0.042361,0.115424,73.301489,89.955022,56.380246,8.214715
2021-12-04 07:33:00,5.793333,825.579167,79.741528,24.724775,7.0,5.882353,44.0,36.974790,24.778187,842.00,350.00,0.030013,0.096589,73.367134,89.955022,58.997050,7.230250
2021-12-04 07:34:00,5.786667,677.605208,111.529095,26.672081,6.5,5.462185,28.5,23.949580,26.860841,657.00,472.75,0.042331,0.183593,96.228441,115.317488,58.852733,14.630706
2021-12-04 07:35:00,5.780000,529.631250,143.316663,28.619386,6.0,5.042017,13.0,10.924370,28.943494,472.00,595.50,0.054648,0.270597,119.089747,140.679953,58.708415,22.031161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-04 22:03:00,6.260000,1092.537500,194.812862,44.162433,28.0,23.529412,72.0,60.504202,44.381416,1177.50,607.00,0.040622,0.178312,57.151129,86.206897,46.047583,12.649581
2021-12-04 22:04:00,6.273333,1161.854167,134.469976,41.109496,22.0,18.487395,62.0,52.100840,41.310218,1212.00,565.00,0.035841,0.119260,53.056762,81.174514,45.907086,8.254712
2021-12-04 22:05:00,6.286667,1231.170833,74.127091,38.056558,16.0,13.445378,52.0,43.697479,38.239021,1246.50,523.00,0.031059,0.060209,48.962394,76.142132,45.766590,3.859843


In [49]:
# viz final chart consisting of 1 minute CGM values/HRV metrics of interest
fig = px.line(
    df,
    x=df.index,
    y=[df['glucose'], df["sdnn"], df["rmssd"], df["pnni_50"]]
)
fig.show()

In [50]:
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("Glucose", "SDNN", "RMSSD", "PNNI-50"))

fig.add_trace(go.Histogram(x=df['glucose']),
              row=1, col=1)

fig.add_trace(go.Histogram(x=df['sdnn']),
              row=1, col=2)

fig.add_trace(go.Histogram(x=df['rmssd']),
              row=2, col=1)

fig.add_trace(go.Histogram(x=df['pnni_50']),
              row=2, col=2)

fig.update_layout(height=500, width=700,
                  title_text="Distribution of HRV metrics and Glucose")

fig.show()


## Observations
No clear patterns can be seen from the data. Continue to prepare the data for further analysis.
Also, from the start of the time series to ~10am, HRV metrics and CGM values are unusually low. Omitting those values might help to improve results.

In [51]:
df = df.iloc[
        df.index.get_loc('2021-12-04 10:00:00'):df.index.get_loc('2021-12-04 22:07:00')
    ]
df

,glucose,mean_nni,sdnn,sdsd,nni_50,pnni_50,nni_20,pnni_20,rmssd,median_nni,range_nni,cvsd,cvnni,mean_hr,max_hr,min_hr,std_hr
date-time,,,,,,,,,,,,,,,,,
2021-12-04 10:00:00,5.346667,603.220833,16.241064,8.998806,0.0,0.000000,4.5,3.781513,9.008226,603.000,94.25,0.014864,0.026697,99.621724,107.677865,92.385139,2.611425
2021-12-04 10:01:00,5.373333,620.404167,21.513656,10.730964,0.0,0.000000,9.0,7.563025,10.731149,620.000,126.50,0.017297,0.034677,96.825331,106.856634,87.209302,3.312531
2021-12-04 10:02:00,5.400000,619.189583,23.331055,11.305472,0.0,0.000000,9.0,7.563025,11.317324,621.125,129.75,0.018280,0.037686,97.039231,107.191758,87.019991,3.656135
2021-12-04 10:03:00,5.406667,617.975000,25.148454,11.879980,0.0,0.000000,9.0,7.563025,11.903499,622.250,133.00,0.019262,0.040695,97.253130,107.526882,86.830680,3.999739
2021-12-04 10:04:00,5.413333,619.908333,39.899877,11.533535,0.0,0.000000,9.0,7.563025,11.594241,628.625,182.25,0.018705,0.064291,97.284207,115.555408,85.256344,6.750296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-04 22:02:00,6.246667,1118.829167,188.084711,46.562012,29.0,24.369748,75.5,63.445378,46.674689,1193.000,639.00,0.041692,0.168343,55.679406,88.420669,45.529418,12.134093
2021-12-04 22:03:00,6.260000,1092.537500,194.812862,44.162433,28.0,23.529412,72.0,60.504202,44.381416,1177.500,607.00,0.040622,0.178312,57.151129,86.206897,46.047583,12.649581
2021-12-04 22:04:00,6.273333,1161.854167,134.469976,41.109496,22.0,18.487395,62.0,52.100840,41.310218,1212.000,565.00,0.035841,0.119260,53.056762,81.174514,45.907086,8.254712


#### Interpreting time series correlation
Refer to:
>https://stats.stackexchange.com/questions/133155/how-to-use-pearson-correlation-correctly-with-time-series
 
Are glucose and RR influenced by both within-series dependence and inter-series dependence or, either or neither of them? Run a pearson and spearman correlation test to find out..

In [52]:
def corr_calc(x, y, type=None):
    return stats.pearsonr(x, y) if type else stats.spearmanr(x, y)
    
pcorr = corr_calc(df['rmssd'], df['glucose'], True)
scorr = corr_calc(df['rmssd'], df['glucose'])
print(f'Pearson: {pcorr}\nSpearman: {scorr}')

Pearson: (-0.05190897829693287, 0.1620671778476934)
Spearman: SpearmanrResult(correlation=-0.09132301435740213, pvalue=0.013768441501476568)


#### Observation
Weak correlation observed with indices of interest. On an intraday basis - little to no correlation for CGM and HRV values.

### Augmented Dickey-Fuller Test (ADF Test)

ADF test is used to test for stationarity of a time-series. A stationary time series does not have trends or seasonal effects.

- Null hypothesis: Time series has a unit root (non stationary)
- Alternate hypothesis: Time series does NOT have a unit root (stationary)

In [53]:
from statsmodels.tsa.stattools import adfuller

In [54]:
X = df['glucose'].values
adf_results = adfuller(X, autolag='AIC')

Some other causuality tests can be tried, i.e. Granger Causality Test to investigate causality between two time series variables.

Let us try to test causality of all variables.

- Null hypothesis: 'RMSSD' does not explain variation in 'glucose'
- Alternative hypothesis: 'RMSSD' has an effect on glucose with a 95% confidence interval that a change in 'RMSSD' causes a response in 'glucose'

In [55]:
from statsmodels.tsa.stattools import grangercausalitytests

In [56]:
df

,glucose,mean_nni,sdnn,sdsd,nni_50,pnni_50,nni_20,pnni_20,rmssd,median_nni,range_nni,cvsd,cvnni,mean_hr,max_hr,min_hr,std_hr
date-time,,,,,,,,,,,,,,,,,
2021-12-04 10:00:00,5.346667,603.220833,16.241064,8.998806,0.0,0.000000,4.5,3.781513,9.008226,603.000,94.25,0.014864,0.026697,99.621724,107.677865,92.385139,2.611425
2021-12-04 10:01:00,5.373333,620.404167,21.513656,10.730964,0.0,0.000000,9.0,7.563025,10.731149,620.000,126.50,0.017297,0.034677,96.825331,106.856634,87.209302,3.312531
2021-12-04 10:02:00,5.400000,619.189583,23.331055,11.305472,0.0,0.000000,9.0,7.563025,11.317324,621.125,129.75,0.018280,0.037686,97.039231,107.191758,87.019991,3.656135
2021-12-04 10:03:00,5.406667,617.975000,25.148454,11.879980,0.0,0.000000,9.0,7.563025,11.903499,622.250,133.00,0.019262,0.040695,97.253130,107.526882,86.830680,3.999739
2021-12-04 10:04:00,5.413333,619.908333,39.899877,11.533535,0.0,0.000000,9.0,7.563025,11.594241,628.625,182.25,0.018705,0.064291,97.284207,115.555408,85.256344,6.750296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-04 22:02:00,6.246667,1118.829167,188.084711,46.562012,29.0,24.369748,75.5,63.445378,46.674689,1193.000,639.00,0.041692,0.168343,55.679406,88.420669,45.529418,12.134093
2021-12-04 22:03:00,6.260000,1092.537500,194.812862,44.162433,28.0,23.529412,72.0,60.504202,44.381416,1177.500,607.00,0.040622,0.178312,57.151129,86.206897,46.047583,12.649581
2021-12-04 22:04:00,6.273333,1161.854167,134.469976,41.109496,22.0,18.487395,62.0,52.100840,41.310218,1212.000,565.00,0.035841,0.119260,53.056762,81.174514,45.907086,8.254712


In [57]:
def normalize(data, percentage=True):
    # divide every data point by the first sample
    norm = data.div(data.iloc[0])
    return norm.mul(100) if percentage else norm

norm = normalize(df)
norm.drop([
    'mean_nni',
    'sdsd',
    'nni_50',
    'pnni_50',
    'nni_20',
    'pnni_20',
    'median_nni',
    'range_nni',
    'cvsd',
    'cvnni',
    'mean_hr',
    'max_hr',
    'min_hr',
    'std_hr'
    ], axis=1, inplace=True)
norm.head()

,glucose,sdnn,rmssd
date-time,,,
2021-12-04 10:00:00,100.000000,100.000000,100.000000
2021-12-04 10:01:00,100.498753,132.464567,119.126113
2021-12-04 10:02:00,100.997506,143.654715,125.633217
2021-12-04 10:03:00,101.122195,154.844864,132.140322
2021-12-04 10:04:00,101.246883,245.672797,128.707262


In [58]:
fig = px.line(
    norm,
    x=norm.index,
    y=[norm['glucose'], norm["sdnn"], norm["rmssd"]]
)
fig.show()